In [2]:
%reload_ext openad.notebooks.styles

<!-- Header banner -->
<div class="banner"><div>Working with the RXN Plugin</div><b>OpenAD <span>Tutorial</span></b></div>

### Table of Contents

1. [Getting Started](#Getting-Started)
    1. [Installation](#Installation)
    1. [Authentication](#Authentication)
    1. [Magic Commands](#Magic-Commands)
    1. [About RXN](#About-RXN)
    1. [Command Documentation](#Command-Documentation)
3. [Forward Reactions](#Forward-Reactions)
   1. [Single Prediction](#Single-Prediction)
   2. [Batch Predictions](#Batch-Predictions)
   3. [Calculating top results at once](#Calculating-top-results-at-once)
4. [Retrosynthesis](#Retrosynthesis)
5. [Interpreting Recipes](#Interpreting-Recipes)
6. [Enriching your Molecules with RXN Results](#Enriching-your-Molecules-with-RXN-Results)

## Getting Started

<div class="alert alert-info">
<b>Note:</b> In order to prevent re-running the same queries as you practice with this Notebook, we'll use the <code>use cache</code> clause in all our examples.
</div>

### Installation
If you haven't already, you can install the plugin directly from its [GitHub repo](https://github.com/acceleratedscience/openad-plugin-rxn#readme).
    
    pip install git+https://github.com/acceleratedscience/openad-plugin-rxn

### Authentication
Before you can work with the RXN plugin, you'll need to sign up for an RXN account and create an API key.<br>We have [detailed instructions in the plugin's README](https://github.com/acceleratedscience/openad-plugin-rxn#authentication). 

### Magic Commands
Magic commands let you interact with the OpenAD shell.
1. `%openad` - Display results directly in your notebook<br>
2. `%openadd` - Store the returned data in a variable

To learn more, check the [OpenAD intro to magic commands](https://github.com/acceleratedscience/openad-toolkit/blob/main/openad/notebooks/magic_commands.ipynb).

### About RXN
To learn about what this plugin does, and to list its available commands, run:

    rxn

In [ ]:
%openad rxn

### Command Documentation

Every command has detailed documentation where you can find everything you need to know, including optional parameters and examples.

To see the documentation of a command, just run the beginning of the command followed by a question mark.

In [ ]:
%openad rxn clear ?

## Forward Reactions

### Single Prediction

Predicting a reaction is as simple as passing a reactions SMILES.

    rxn predict reaction '<smiles>.<smiles>'

In [ ]:
# Hydrochloric acid + N-propylpropanamide + Oxygen + Water
%openad rxn predict reaction 'Cl.CCC(=O)NCCC.O' use cache

### Rich visualization

Because things can become hard to read when your smiles strings are long, we provide a richer visualization.<br>
This allows you to better see the structure of yoru reaction.

This same clause is available for batch predictions.

In [ ]:
# Rich visualisation
%openad rxn predict reaction 'CC(C)(c1ccccn1)C(CC(=O)O)Nc1nc(-c2c[nH]c3ncc(Cl)cc23)c(C#N)cc1F.Cl.CCC(=O)NCCC.O' rich use cache

### Batch Predictions

Batch predictions can be done from a list of SMILES, or a file or dataframe containing lists of smiles.

#### From a List
    
    rxn predict reactions from list ['<smiles>.<smiles>',...]

In [ ]:
# Bromine + 2-anthracen-1-ylethanol / anthracene
%openad rxn predict reactions from list ['BrBr.c1ccc2cc3ccccc3cc2c1', 'BrBr.c1ccc2cc3ccccc3cc2c1CCO'] use cache

#### From a DataFrame

    rxn predict reactions from dataframe <dataframe_name>
    
Your DataFrame should have a "Reactions" column.

In [ ]:
# Create a Pandas DataFrame with reaction SMILES
reactions = ["BrBr.c1ccc2cc3ccccc3cc2c1", "BrBr.c1ccc2cc3ccccc3cc2c1CCO"]
df = py.DataFrame(reactions, columns=["Reactions"])

In [ ]:
# Predict reactions
%openad rxn predict reactions from dataframe df use cache

#### From a File

    rxn predict reactions from file '<filename.csv>'
    
When using a CSV file, it should contain a "Reactions" column, just like with a dataframe. Alternatively, you can simply use a text file with one reaction per line.

For the purpose of this demo, we'll store both a .txt a .csv file with reactions in your workspace.

In [ ]:
import pandas as py

# Prep
reactions = ["BrBr.c1ccc2cc3ccccc3cc2c1", "BrBr.c1ccc2cc3ccccc3cc2c1CCO"]
cmd_pointer = %openadd cmd_pointer
workspace_path = cmd_pointer.workspace_path()
csv_file_path = f"{workspace_path}/rxn_demo_reactions.csv"
text_file_path = f"{workspace_path}/rxn_demo_reactions.txt"

# Store reactions in a CSV file in your workspace
df = py.DataFrame(reactions, columns=["Reactions"])
df.to_csv(csv_file_path)

# Store reactions in a text file in your workspace
with open(text_file_path, "w") as file:
    for item in reactions:
        _ = file.write(f"{item}\n");

In [ ]:
# Inspect the files we just created
%cat {csv_file_path}
print()
%cat {text_file_path}

In [ ]:
# Predict reactions from a CSV file
%openad rxn predict reactions from file 'rxn_demo_reactions.csv' use cache

In [ ]:
# Predict reactions from a text file
%openad rxn predict reactions from file 'rxn_demo_reactions.txt' use cache

### Calculating top results at once

Sometimes one result per reaction may not be enough. You may want to see a list of most likely outcomes ranked by confidence.

To do this, you can simply pass the `topn` parameter and set it to however many outcomes you want to see.

This works for both single predictions and batch predictions.

In [ ]:
%openad rxn predict reaction 'BrBr.c1ccc2cc3ccccc3cc2c1' using (topn=3) use cache

In [ ]:
%openad rxn predict reactions from list ['BrBr.c1ccc2cc3ccccc3cc2c1', 'BrBr.c1ccc2cc3ccccc3cc2c1CCO'] using (topn=5) use cache

## Retrosynthesis

    rxn predict retro '<smiles>'

Finding the retrosynthesis route of a molecule is again as simple as providing its SMILES.

There's a number of options available, like `max_steps` or `exclude_substructures`.<br>
To see the full list of options you can consult the command's documentation.

In [ ]:
%openad rxn predict retro ?

You'll notice that we are using variable substitution below to provide the command with the SMILES molecule.

In [ ]:
smiles = "CC(C)(c1ccccn1)C(CC(=O)O)Nc1nc(-c2c[nH]c3ncc(Cl)cc23)c(C#N)cc1F"
%openad rxn predict retro {smiles} using (max_steps=5) use cache

### Rich visualization
Because a reaction tree is not linear, we provide a richer visualization. This makes it easier to piece together the structure of the tree, but takes up more space

In [ ]:
%openad rxn predict retro {smiles} using (max_steps=5) rich use cache

### Returning data
Because a reaction tree is not linear, the data is returned in JSON format by default.

If you wish to process your data in a table instead, you can use the `return df` clause to request the data as a DataFrame.

In [ ]:
# Return data as JSON
json_data = %openadd rxn predict retro {smiles} using (max_steps=5) use cache
json_data

In [ ]:
# Return data as DataFrame
df = %openadd rxn predict retro {smiles} using (max_steps=5) use cache return df
df

## Interpreting Recipes

RXN also lets you interpret a textual description of a process for performing a chemical reaction and spit it back as a set of defined steps.

#### From a paragraph

    rxn interpret recipe '<recipe>'

In [ ]:
# Interpret recipe from a text paragraph
%openad rxn interpret recipe 'To a stirred solution of 7-(difluoromethylsulfonyl)-4-fluoro-indan-1-one \
(110 mg, 0.42 mmol) in methanol (4 mL) was added sodium borohydride (24 mg, 0.62 mmol). \
The reaction mixture was stirred at ambient temperature for 1 hour.'

#### From a file
    
    rxn interpret recipe '<recipe.txt>'

For the purpose of this demo, we'll store a `rxn_demo_recipe.txt` file in your workspace.

In [ ]:
# Prep
cmd_pointer = %openadd cmd_pointer
workspace_path = cmd_pointer.workspace_path()
recipe_file_path = f"{workspace_path}/rxn_demo_recipe.txt"

recipe = "To a stirred solution of 7-(difluoromethylsulfonyl)-4-fluoro-indan-1-one \
(110 mg, 0.42 mmol) in methanol (4 mL) was added sodium borohydride (24 mg, 0.62 mmol). \
The reaction mixture was stirred at ambient temperature for 1 hour."

# Store recipe in a text file in your workspace
with open(recipe_file_path, "w") as file:
    _ = file.write(recipe);

In [ ]:
# Inspect the file we just created
%cat {recipe_file_path}

In [ ]:
# Interpret recipe from a text file
%openad rxn interpret recipe 'rxn_demo_recipe.txt'

## Enriching your Molecules with RXN Results

After running an RXN query, you can add the results to the related molecules in your molecule working set.

    enrich molecules with analysis

Commands that are supported by this functionality:
- `rxn predict retrosynthesis ...`
- `rxn predict reaction(s) ...`

In [ ]:
# Clear any previously stored results
%openad clear analysis cache

# Empty your molecule working set
%openad clear mols

In [ ]:
# Run retrosynthesis query (using %openadd to skip the printout)
smiles = "CC(C)(c1ccccn1)C(CC(=O)O)Nc1nc(-c2c[nH]c3ncc(Cl)cc23)c(C#N)cc1F"
_ = %openadd rxn predict retro '{smiles}' using (max_steps=5)

# Add the relevant molecule to your molecule working set (MWS)
%openad add molecule {smiles}

# Enrich the MWS with the RXN result
%openad enrich molecules with analysis

# Display the molecule to see the result (scroll down to analysis).
# From here you can export the molecule to a new file.
%openad show molecule {smiles}